In [7]:
import micromagneticdata as mdata
import discretisedfield as df
import os

from multiprocessing import Pool
import pandas as pd


import ast

In [2]:
name = 'mbp_l_1500_w_100_ht_10_ixxxx'

In [11]:
def locate(filename):
    m = df.Field.from_file(filename)

    info = df.tools.count_bps(m)

    coarse = ast.literal_eval(info['bp_pattern_x'])[0][1] * 2.5e-9
    info['loc_x_coarse'] = coarse

    dx = 20e-9
    m_small = m.sel(x=(coarse - dx, coarse + dx))
    
    em_div = df.tools.emergent_magnetic_field(m_small).div

    loc = (em_div * em_div.mesh.coordinate_field()).integrate() / em_div.integrate()

    info['loc_fine'] = loc


    return info

In [7]:
for j in [5e10, 1e11, 1.5e11, 2e11, 3e11, 5e11, 7e11, 1e12, 2e12]:
    drive = mdata.Data(name=name, dirname=f'simulations/J_{j:g}')[0]
    with Pool(len(os.sched_getaffinity(0))) as p:  # use all available CPUs
        res = p.map(locate, drive._step_files)


    converted = {
        'bp_number': [elem['bp_number'] for elem in res],
        'bp_number_hh': [elem['bp_number_hh'] for elem in res],
        'bp_number_tt': [elem['bp_number_tt'] for elem in res],
        'loc_x_coarse': [elem['loc_x_coarse'] for elem in res],
        'loc_x_fine': [elem['loc_fine'][0] for elem in res],
        'loc_y_fine': [elem['loc_fine'][1] for elem in res],
    }
    
    pd.DataFrame(converted).to_csv(f"bp-position-j-{J * 1e9:g}.csv", index=False)